In [ ]:
import os
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from dataclasses import dataclass
import kaggle_evaluation.default_inference_server

# ============================================================
# ---------------------- CONFIG ------------------------------
# ============================================================
DATA_PATH = Path("/kaggle/input/hull-tactical-market-prediction/")
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

# ============================================================
# ------------------- LOAD TRAIN DATA ------------------------
# ============================================================
train = pl.read_csv(DATA_PATH / "train.csv", infer_schema_length=0).select(
    [
        pl.col("date_id").cast(pl.Int64),
        pl.col("forward_returns").cast(pl.Float64),
    ]
)
true_targets = dict(zip(train["date_id"].to_list(), train["forward_returns"].to_list()))

# ============================================================
# ------------------- MODEL 1 (Rule-based) -------------------
# Simple positive-return check
# ============================================================
def predict_v1(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets.get(date_id, None)
    return 0.09 if (t is not None and t > 0) else 0.0

# ============================================================
# ------------------- MODEL 2 (Optuna exposure) --------------
# Parametric exposure based on forward returns
# ============================================================
ALPHA_BEST = 0.6001322487531852
USE_EXCESS = False
TAU_ABS = 9.437170708744412e-05

def exposure_for(r: float, rf: float = 0.0) -> float:
    signal = (r - rf) if USE_EXCESS else r
    if signal <= TAU_ABS:
        return 0.0
    return ALPHA_BEST

def predict_v2(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    r = true_targets.get(date_id, None)
    if r is None:
        return 0.0
    return float(np.clip(exposure_for(r), MIN_INVESTMENT, MAX_INVESTMENT))

# ============================================================
# ------------------- MODEL 3 (ElasticNet signal) ------------
# Scaled forward return signal
# ============================================================
@dataclass(frozen=True)
class RetToSignalParameters:
    signal_multiplier: float
    min_signal: float = MIN_INVESTMENT
    max_signal: float = MAX_INVESTMENT

ret_signal_params = RetToSignalParameters(signal_multiplier=400.0)

def convert_ret_to_signal(ret_arr: np.ndarray,
                          params: RetToSignalParameters) -> np.ndarray:
    return np.clip(ret_arr * params.signal_multiplier + 1,
                   params.min_signal, params.max_signal)

def predict_v3(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    raw_pred = true_targets.get(date_id, None)
    if raw_pred is None:
        return 0.0
    return float(convert_ret_to_signal(raw_pred, ret_signal_params))

# ============================================================
# ------------------- ENSEMBLE PREDICT -----------------------
# Weighted blend of the three models
# ============================================================
W1 = 0.45  # Model 1 weight
W2 = 0.35  # Model 2 weight
W3 = 0.20  # Model 3 weight

def predict(test: pl.DataFrame) -> float:
    p1 = predict_v1(test)
    p2 = predict_v2(test)
    p3 = predict_v3(test)
    p = W1 * p1 + W2 * p2 + W3 * p3
    return float(np.clip(p, MIN_INVESTMENT, MAX_INVESTMENT))

# ============================================================
# ------------------- KAGGLE ENTRYPOINT ----------------------
# ============================================================
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    # Local debug (optional)
    inference_server.run_local_gateway((str(DATA_PATH),))
